In [1]:
import numpy as np 
import pandas as pd
import os
from Dataset.Utils import prepare_data_range, read_fasta
from Dataset.Resample import resample

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from diamondonpy import Diamond
from pathlib import Path

diamond = Diamond()

db_path = Path("/mnt/d/ML/Kaggle/CAFA6-new/uniprot/uniprot_sprot.dmnd")
query_path = Path("/mnt/d/ML/Kaggle/CAFA6/cafa-6-protein-function-prediction/train/train_sequences.fasta") 

results_df = diamond.blastp(
    db=str(db_path),    
    query=str(query_path),
    outfmt="6 qseqid sseqid pident evalue bitscore qcovhsp",
    evalue=1e-3,
    max_target_seqs=500,
)


In [3]:
results_df.head()

,qseqid,sseqid,pident,evalue,bitscore,qcovhsp
0,sp|A0A0C5B5G6|MOTSC_HUMAN,sp|A0A0C5B5G6|MOTSC_HUMAN,100.0,7.220000e-06,38.9,100
1,sp|A0JNW5|BLT3B_HUMAN,sp|A0JNW5|BLT3B_HUMAN,100.0,0.000000e+00,2818.0,100
2,sp|A0JNW5|BLT3B_HUMAN,sp|A2RSJ4|BLT3B_MOUSE,83.6,0.000000e+00,2352.0,99.8
3,sp|A0JNW5|BLT3B_HUMAN,sp|Q6NRZ1|BLT3B_XENLA,59.2,0.000000e+00,1578.0,100
4,sp|A0JNW5|BLT3B_HUMAN,sp|Q6BDS2|BLT3A_HUMAN,40.9,3.310000e-312,925.0,99.7


In [4]:
results_df['sseqid'].nunique()

469658

In [5]:

print(results_df.shape)
results_df.head()
# Extract IDs from pipe-delimited format (e.g., sp|P12345|PROT_HUMAN -> P12345)
results_df['qseqid'] = results_df['qseqid'].str.split('|').str[1]
results_df['sseqid'] = results_df['sseqid'].str.split('|').str[1]

print(results_df.shape)
results_df.head()
results_df[['evalue', 'bitscore']].describe()

results_df.drop(results_df[results_df['qseqid'] == results_df['sseqid']].index, inplace=True)
results_df.shape
results_df.groupby('qseqid').agg(num_hits=('sseqid', 'count')).describe()

(6599605, 6)
(6599605, 6)


,num_hits
count,77035.000000
mean,84.604167
std,133.712480
min,1.000000
25%,7.000000
50%,24.000000
75%,87.000000
max,499.000000


In [9]:
results_df.to_csv("/mnt/d/ML/Kaggle/CAFA6-new/uniprot/train_vs_swissprot.tsv", sep="\t", index=False)

In [ ]:
import pandas as pd
from pathlib import Path
from collections import defaultdict

def parse_gaf_to_dict(gaf_path: str) -> dict:
    """
    Parse UniProt GOA GAF file into protein_id -> list of GO terms.
    
    GAF columns:
    0: DB, 1: DB_Object_ID (protein), 2: DB_Object_Symbol, 3: Qualifier,
    4: GO_ID, 5: DB:Reference, ...
    
    We want columns 1 (protein) and 4 (GO term).
    """
    protein_to_go = defaultdict(set)
    
    with open(gaf_path, 'r') as f:
        for line_num, line in enumerate(f, 1):
            # Skip comment lines
            if line.startswith('!'):
                continue
            
            parts = line.rstrip('\n').split('\t')
            if len(parts) < 5:
                continue
            
            protein_id = parts[1]  # DB_Object_ID (e.g., "P12345")
            go_id = parts[4]       # GO_ID (e.g., "GO:0005524")
            
            # Optionally filter by evidence or other criteria
            # evidence = parts[6] if len(parts) > 6 else None
            # if evidence in ['IEP', 'IEA']:  # filter if desired
            #     continue
            
            protein_to_go[protein_id].add(go_id)
            
            if line_num % 100000 == 0:
                print(f"Parsed {line_num:,} lines, {len(protein_to_go):,} proteins...")
    
    # Convert sets to lists for easier handling
    protein_to_go = {k: list(v) for k, v in protein_to_go.items()}
    print(f"Total: {len(protein_to_go):,} proteins with GO terms")
    return protein_to_go

# Parse the GAF file (this takes a few minutes for the full file)
gaf_path = Path("/mnt/d/ML/Kaggle/CAFA6-new/data/goa_uniprot_all.gaf")
protein_to_go = parse_gaf_to_dict(str(gaf_path))

# Example: check a few proteins
sample_protein = list(protein_to_go.keys())[0]
print(f"\n{sample_protein}: {protein_to_go[sample_protein][:5]}...")
